In [1]:
import sys
sys.path.append("/reg/neh/home/haoyuan/Documents/my_repos/DiffusionMap")

from holoviews.operation.datashader import datashade
import dask.array
import h5py
import numpy as np
import holoviews as hv
from pDiffusionMap import visutil, visabbr

hv.extension('bokeh', width=90, logo=False)


# [USER] Initialize datasource and extract some samples

In [9]:
# Raw data file list
raw_data_path = '../input/file_list.txt'

# Sampling number 
sample_number = 500


# [AUTO] Sample the index and dataset

In [4]:
(data_source,
 pattern_shape,
 global_index_map,
 sampled_index,
 sampled_patterns) = visabbr.load_data_and_get_samples(input_txtfile=raw_data_path,
                                                       sample_number=sample_number)


It takes 53.25032091140747 seconds to sample 2000 patterns.


# [USER] Specify visualization parameters

In [12]:
#  Load eigenvectors 
with h5py.File('../output/eigensystem_2018_08_02_16_04_43.h5', 'r') as h5file:
    eigenvectors = np.array(h5file['eigenvectors'])

# Eigenvectors to show
dim0 = 0
dim1 = 1

# Values range of the images to show
value_range = [0, 100]

# Number of patterns to show along each axis
num = 2

# Diffusion Map diemsion along each axis in pixel number
length = 600

# Dimension of the sampled patterns. Please use this default value
sample_size = int(length * 0.9 / num)

# [AUTO] Setup Initialization

In [ ]:
(points_all, background,
 sampled_points, sampled_positions,
 check, select, path_stream) = visabbr.get_background_sample_and_streams(data_source=data_source,
                                                                         eigens=eigenvectors,
                                                                         dim0=dim0, dim1=dim1,
                                                                         length=length,
                                                                         sample_number=sample_number,
                                                                         sampled_index=sampled_index)


#########################################################
# [Auto] Define actions
#########################################################
def sample_from_selected_region(index):
    """
    This is the action when the user select a rectangular region to inspect.
    :param index: 
    :return: 
    """
    return visutil.assemble_patterns(col_num=num,
                                     row_num=num,
                                     index=index,
                                     data_holder=sampled_patterns,
                                     data_shape=pattern_shape,
                                     value_range=value_range,
                                     height=sample_size,
                                     width=sample_size)


# Handle for box selection
handle_check = hv.DynamicMap(sample_from_selected_region, streams=[check])


# [AUTO] Visualization

In [14]:
# Create the final diagram
layout = (background * sampled_points * select + handle_check.collate()).options(shared_axes=False)
layout


:Layout
   .DynamicMap.I :DynamicMap   []
      :Overlay
         .RGB.I      :RGB   [x,y]   (R,G,B,A)
         .Points.I   :Points   [x,y]
         .Polygons.I :Polygons   [x,y]
   .GridSpace.I  :GridSpace   [X,Y]
      :DynamicMap   []
         :Image   [x,y]   (z)

# [USER] Save selected polygon

In [13]:
output_address = '../output/browser_1_selected_region.npy'

index, points = visutil.save_selected_region(stream_holder=path_stream,
                                             data_holder=data_all_coor,
                                             output=output_address,
                                             return_selected_region=True)


In [15]:
# Datashade the selected region
datashade(hv.Points(points), dynamic=True)


:DynamicMap   []
   :RGB   [x,y]   (R,G,B,A)